# Using Monte Carlo Tree Search for your Fantasy Football draft
Two more months till the next American Football season kicks off, which means Fantasy Football players around the world are preparing for their upcoming league draft. In this blog we will use the Monte Carlo Tree Search algorithm to optimize our next pick in a typical snake draft. If you are not familiar with Fantasy Football or snake drafts, you can find a good introduction at ESPN's Fantasy Football 101. We will look at a standard PPR scoring league on ESPN consisting of ten teams, but these settings should be easy to change to fit your own league's.

In [29]:
import pandas as pd

In [30]:
class DraftState:
    def __init__(self, rosters, turns, freeagents, playerjm=None):
        self.rosters = rosters
        self.turns = turns
        self.freeagents = freeagents
        self.playerJustMoved = playerjm

In [31]:
def GetResult(self, playerjm):
    """ Get the game result from the viewpoint of playerjm.
    """
    pos_wgts = {
        ["QB"]: [.6, .4],
        ["WR"]: [.7, .7, .4, .2],
        ["RB"]: [.7, .7, .4, .2],
        ["TE"]: [.6, .4],
        ["RB", "WR", "TE"]: [.6, .4],
        ["D"]: [.6, .3, .1],
        ["K"]: [.5, .2, .2, .1]
    }

    if playerjm is None: return 0

    roster = self.rosters[playerjm].sort_values("points", ascending=False)
    result = 0
    for roster_pos, pts in zip(roster["position"], roster["points"]):
        max_wgt, _, max_pos = max(
            ((wgts[0], len(lineup_pos), lineup_pos) for lineup_pos, wgts in pos_wgts.items()
                if roster_pos in lineup_pos),
            default=0)
        if max_wgt > 0:
            result += max_wgt * pts
            old_wgts = pos_wgts[max_pos]
            if len(old_wgts) > 1:
                pos_wgts[max_pos] = old_wgts[1:]
            else:
                pos_wgts.pop(max_pos)
                
    for pos, wgts in pos_wgts.items():
        fa = self.freeagents
        res += fa.points[fa.position.isin(pos)].head(3).mean()
        
DraftState.GetResult = GetResult

In [37]:
def GetMoves(self):
    """ Get all possible moves from this state.
    """
    POS_MAX = {"QB": 2, "WR": 6, "RB": 6, "TE": 2, "D": 2, "K": 1}

    if len(self.turns) == 0: return []

    roster = self.rosters[self.turns[0]]
    moves = [pos for pos, max_ in POS_MAX if sum(roster.position == pos) < max_]
    return list(moves)

DraftState.GetMoves = GetMoves

In [4]:
nfl_players = pd.read_csv("nfl_players.csv", index_col=0)

In [17]:
roster = pd.DataFrame(columns=["name", "team", "position", "points"])